<a href="https://colab.research.google.com/github/RoshanKumar854/ML-Projects/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [26]:
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [27]:
train_df = train_df.drop(['id'], axis=1)

In [28]:
train_df_A = train_df.drop(['comment_text'], axis=1)
train_df_A = train_df_A.idxmax(axis=1)

In [29]:
train_df_B = train_df["comment_text"]

In [30]:
train_df = pd.concat([train_df_A, train_df_B], axis=1)
train_df.rename(columns={0 :'label'}, inplace=True)
train_df

,label,comment_text
0,toxic,Explanation\nWhy the edits made under my usern...
1,toxic,D'aww! He matches this background colour I'm s...
2,toxic,"Hey man, I'm really not trying to edit war. It..."
3,toxic,"""\nMore\nI can't make any real suggestions on ..."
4,toxic,"You, sir, are my hero. Any chance you remember..."
...,...,...
159566,toxic,""":::::And for the second time of asking, when ..."
159567,toxic,You should be ashamed of yourself \n\nThat is ...
159568,toxic,"Spitzer \n\nUmm, theres no actual article for ..."
159569,toxic,And it looks like it was actually you who put ...


In [31]:
label = train_df.label.factorize()
label


(array([0, 0, 0, ..., 0, 0, 0]),
 Index(['toxic', 'insult', 'obscene', 'identity_hate', 'threat'], dtype='object'))

In [32]:
text = train_df.comment_text.values
text = text.astype(str)

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(text)


In [34]:
text = tokenizer.texts_to_sequences(text)

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(text, maxlen=200)
vocab_size = 2000


In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(5, activation="sigmoid"))
model.compile(loss="SparseCategoricalCrossentropy", optimizer="adam", metrics=["SparseCategoricalAccuracy"])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 32)           64000     
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 200, 32)          0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 50)                16600     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 255       
                                                                 
Total params: 80,855
Trainable params: 80,855
Non-trainable params: 0
__________________________________________________

In [ ]:
history = model.fit(padded_sequence, label[0] ,validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
3252/3990 [=======================>......] - ETA: 16:32 - loss: 0.0508 - sparse_categorical_accuracy: 0.9936

In [ ]:
model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)

In [ ]:
plt.plot(history.history["accuracy"], label="acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.legend()
plt.show()

plt.savefig("Accuracy plot.png")


In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")

plt.legend()
plt.show()

plt.savefig("Loss plot.png")


In [ ]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=500)
    prediction = int(model.predict(tw).round().item())
    print("label: ", label[1][prediction])


test_sentence1 = "I enjoyed my journey on this flight."
predict_sentiment(test_sentence1)

test_sentence2 = "This is the worst flight experience of my life!"
predict_sentiment(test_sentence2)

